In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('weights/best.pt')  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model(["img_sample/extract840.jpg", 'img_sample/extract841.jpg'])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs

# Inference Arguments

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('weights/best.pt')

# Run inference on 'bus.jpg' with arguments
result = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.4)

# Boxes

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('weights/best.pt')

# Run inference on an image
results = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.5)  # results list

# View results
for r in results:
    print(r.boxes)  # print the Boxes object containing the detection bounding boxes

# Masks

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n-seg Segment model
model = YOLO('weights/best.pt')

# Run inference on an image
results = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.5)  # results list

# View results
for r in results:
    print(r.masks)  # print the Masks object containing the detected instance masks

# Keypoints

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n-pose Pose model
model = YOLO('weights/best.pt')

# Run inference on an image
results = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.5)  # results list

# View results
for r in results:
    print(r.keypoints)  # print the Keypoints object containing the detected keypoints


# Probs

In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n-cls Classify model
model = YOLO('weights/best.pt')

# Run inference on an image
results = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.5)  # results list

# View results
for r in results:
    print(r.probs)  # print the Probs object containing the detected class probabilities


# Plotting Results

In [2]:
from PIL import Image
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('weights/best.pt')

# Run inference on an image
results = model.predict('img_sample/extract840.jpg', save=False, imgsz=1024, conf=0.5)  # results list

# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    # im.save('results.jpg')  # save image

FileNotFoundError: [Errno 2] No such file or directory: 'weights\\best.pt'

# Streaming Source for-loop

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from ouster import client
from ouster import pcap
# from ouster.sdk import viz
from ouster.client import Scans
from ouster.client.data import LidarScan
from ultralytics import YOLO
from contextlib import closing



# Load the YOLOv8 model
model = YOLO('weights/best_x_0_92mAP50.pt')

# Paths to pcap and json files
# test_2_wilanow
metadata_path = "C:/Users/szyme/Ouster/data/PKR_test2/test_2_wilanow.json"
pcap_path = "C:/Users/szyme/Ouster/data/PKR_test2/test_2_wilanow.pcap"

# Making PacketSource from data
with open(metadata_path, 'r') as f:
    metadata = client.SensorInfo(f.read())


fps = int(str(metadata.mode)[-2:])
width = int(str(metadata.mode)[:4])
height = int(str(metadata.prod_line)[5:])

pcap_file = pcap.Pcap(pcap_path, metadata)

with closing(client.Scans(pcap_file)) as scans:

    save_path = "C:/Users/szyme/Ouster/YOLOv8/results_mp4"
    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for scan in scans:
        # Create signal image
        sig_field = scan.field(client.ChanField.SIGNAL)
        sig_destaggered = client.destagger(pcap_file.metadata, sig_field)

        # Define a scaling factor based on the values (adjust this as needed)
        scaling_factor = 0.004
        # Lower the elements based on their values
        lowered_arr = sig_destaggered / (0.5 + scaling_factor * sig_destaggered) # BASIC
        # Convert to uint8
        signal_image = lowered_arr.astype(np.uint8)
        combined_img = np.dstack((signal_image, signal_image, signal_image))
        # print(signal_image.shape)

        # Create range image (for localization, distance measurement)
        range_field = scan.field(client.ChanField.RANGE)
        range_image = client.destagger(pcap_file.metadata, range_field)


        # Predict image
        results = model.predict(combined_img, save=False, imgsz=1024, conf=0.5)
        # results = model.track(source=combined_img, persist=True, iou=0.5, conf=0.4, tracker='botsort.yaml')
            
        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Stream results
        if True:
            cv2.imshow("YOLOv8 Tracking", annotated_frame)
            cv2.waitKey(1)  # 1 millisecond

        vid_writer.write(annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    vid_writer.release()
    cv2.destroyAllWindows()

# Functions to algorithm

Detekcja sytuacji niebezpiecznych. Środkiem układu współrzędnych kartezjańskich jest środek lidaru. Oś `x` przecina środek lidaru i jej współrzędna rośnie w kierunku przodu pojazdu. Oś `y` rośnie na lewo od osi `x`. Wartość współrzędnej `z` rośnie do góry.

Aby dokonać detekci sytuacji niebezpiecznych bazuję na wektorach prędkości obiektów. Powstaja one na podstawie kolejnych klatek obrazu (punkt pierwszy i n-ty z współrzędnymi `xyz`). Wyznaczam prostą przechodzącą przez te dwa punkty. Jeżeli przetnie ona niewielki obszar wokół pojazdu, oznacza to, że pieszy zderzy się z pojazdem

In [25]:
import time
import numpy as np

EPS = np.finfo(float).eps

def linear_func_coefficients(p1, p2):
    """
    Funkcja zwracająca współczynniki prostej przechodzącej przez 2 punkty p1 i p2
    """

    if (p2[0] - p1[0]) == 0:
        a = (p2[1]-p1[1])/(EPS)
    else:
        a = (p2[1]-p1[1])/(p2[0]-p1[0])
    b = p1[1] - a*p1[0]
    return a, b

def breaking_distance(v):
    """ 
    Parameters:
        v - velocity
        t - time of react
        f - coefficient of friction (0.7 on dry asphalt, 0.3 on wet)
        G - slope of the road expressed in decimal value
    Return:
        s - distance of breaking, formula AASHTO
    """
    t = 0
    f = 0.2
    G = 0

    s = (0.278 * t * v) + v**2 / (254 * (f + G))
    return s


def calculate_distance(track, id):
    distance = math.sqrt(track[-1][0]**2 + track[-1][1]**2) - 1.5 # dystans w lini prostej
    # distance = track[-1][1] # y
    return distance

def x_speed_func(p1, p2, diff_frames):
    """
        v = s / t
    """
    # x_speed = (p1[0] - p2[0])/(0.1)
    x_speed = (p1[0] - p2[0])/(0.05*diff_frames)
    return x_speed

def dangerous_situation_func(track, id):
    """
    Frame rate: 10 Hz

    """
    distance = calculate_distance(track, id)
    # Gdy jest możliwe wyznaczenie prostej (detekcja człowieka i przypisanie ID)
    if len(track) > 1:
        distance = math.sqrt(track[-1][0]**2 + track[-1][1]**2) - 1.5
        y_distance = track[-1][1]
        x_distance = track[-1][0]

        p1, p2 = track[0], track[-1]
        a, b = linear_func_coefficients(p1, p2)
        x_speed = x_speed_func(p1, p2, len(track)-1)
        s_break = breaking_distance(abs(x_speed))

        if a == 0:
            a = EPS

        acc_1_x = (2-b)/a
        acc_2_x = (-2-b)/a

        # Jeżeli prosta wyznaczona z wektora prędkości pieszego przetnie obszar samochodu + dodatkowa przestrzeń 
        if (acc_1_x < 5 and acc_1_x > 0) or (acc_1_x > 5 and acc_2_x < 0) or (acc_2_x > 5 and acc_1_x < 0) or (acc_2_x < 5 and acc_2_x > 0):
            # Człowiek się oddala
            if x_speed < 0 :
                if distance < 5 :
                    return "BE CAREFUL", (0, 155, 100), distance
                else:
                    return "GO AHEAD", (0, 255, 0), distance
            elif x_speed > 80:
                return "MISTAKE", (255, 0, 0), distance
            else:
                if 0 < x_distance < 30:
                    if -7 < y_distance < 7:
                        if x_speed < 3:
                            return "BE CAREFUL", (0, 155, 100), distance
                        elif x_speed < 8:
                            return "SLOW DOWN", (0, 100, 155), distance
                        else:
                            print(f"BREAK: {id}\n")
                            return "BREAK", (0, 0, 255), distance
                    else:
                        return "GO AHEAD", (0, 255, 0), distance
                else: 
                    return "GO AHEAD", (0, 255, 0), distance          
        # Gdy prosta nie przecina obszaru wokół samochodu:
        else:
            if 0 < x_distance < 20: 
                # Jeżeli wartość odległości współrzędnej y od obiektu jest mniejsza niż 1 m: ZWOLNIJ 
                if y_distance < 2 and y_distance > -2:
                    return "BE CAREFUL", (0, 100, 155), distance
                # W przeciwnym razie: JEDŹ DALEJ
                else:
                    return "GO AHEAD", (0, 255, 0), distance
            else:
                return "GO AHEAD", (0, 255, 0), distance
    # Brak detekcji ludzi: JEDŹ DALEJ
    else :
        return "GO AHEAD", (0, 255, 0), distance


def correct_track(track):
    if len(track) == 3:
        movement = track[-1] - track[-2]
        previous_movement = track[-2] - track[-3]

        # jeśli aktualny ruch różni się znacznie od poprzedniego, dokonuje korekcji
        if any(movement > previous_movement*3 ):
            track[-1] = track[-2] + previous_movement

In [29]:
import cv2
import math
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from ouster import client
from ouster import pcap
# from ouster.sdk import viz
from ouster.client import Scans
from ouster.client.data import LidarScan
from ultralytics import YOLO
from contextlib import closing
from collections import defaultdict



# Load the YOLOv8 model
model = YOLO('weights/best_s.pt')

# Paths to pcap and json files
metadata_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.json"
pcap_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.pcap"

# Making PacketSource from data
with open(metadata_path, 'r') as f:
    metadata = client.SensorInfo(f.read())


fps = int(str(metadata.mode)[-2:])
width = int(str(metadata.mode)[:4])
height = int(str(metadata.prod_line)[5:])

pcap_file = pcap.Pcap(pcap_path, metadata)

xyzlut = client.XYZLut(metadata) #call cartesian lookup table

# Store the track history
track_history = defaultdict(lambda: [])

with closing(client.Scans(pcap_file)) as scans:

    save_path = "C:/Users/szyme/Ouster/YOLOv8/results_mp4"
    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for scan in scans:
        # Create signal image
        sig_field = scan.field(client.ChanField.SIGNAL)
        sig_destaggered = client.destagger(pcap_file.metadata, sig_field)
    
        # Define a scaling factor based on the values (adjust this as needed)
        scaling_factor = 0.004
        # Lower the elements based on their values
        lowered_arr = sig_destaggered / (0.5 + scaling_factor * sig_destaggered) # BASIC
        # Convert to uint8
        signal_image = lowered_arr.astype(np.uint8)
        combined_img = np.dstack((signal_image, signal_image, signal_image))

        # Create range image (for localization, distance measurement)
        range_field = scan.field(client.ChanField.RANGE)
        range_image = client.destagger(pcap_file.metadata, range_field)

        # xyz_destaggered = xyzlut(range_field)
        xyz_destaggered = client.destagger(metadata, xyzlut(scan)) #to adjust for the pixel staggering that is inherent to Ouster lidar sensor raw data


        # Predict and track
        # results = model.predict(combined_img, save=False, imgsz=1024, conf=0.5)
        results = model.track(source=combined_img, persist=True, imgsz=1024, tracker='bytetrack.yaml')

        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)

        if (results[0].boxes.id == None):
            ids = ''
        else:
            ids = results[0].boxes.id.cpu().numpy().astype(int)

        OUTPUT = "GO AHEAD"
        color = (0, 255, 0)
        
        for box, id in zip(boxes, ids):

            center_x, center_y = int((box[0] + box[2])/2), int((box[1] + box[3])/2)

            xyz_val = xyz_destaggered[(center_y, center_x)] #get the (x,y,z) coordinates with the lookup table 
            # xyz_vals = xyz_destaggered[center_y-1:center_y+2, center_x-1:center_x+2]
            # xyz_val_mean = np.mean(np.mean(xyz_vals, axis=0), axis=0)

            track = track_history[id] #save the (x,y,z) coordinates for distance calculation

            track.append(xyz_val)

            # check if not mistake
            correct_track(track)

            # result will be one of: BREAK, SLOW_DOWN, BE_CAREFUL, GO_AHEAD 
            OUTPUT, color, distance = dangerous_situation_func(track, id)

            if len(track) > 3:
                track.pop(0)


            cv2.rectangle(combined_img, (box[0], box[1]), (box[2], box[3]), color, 2)
            cv2.rectangle(combined_img, (box[0], box[1]+2), (box[0]+160, box[1]-12), (255, 255, 255), -1)
            cv2.putText(combined_img, f"Id {id}; dist: {distance:0.2f} m", (box[0], box[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # if isinstance(relative_speed, float): 
        #     cv2.putText(combined_img, f"{OUTPUT:0.2f}", (400, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        # elif isinstance(relative_speed, str):
        #     cv2.putText(combined_img, f"{OUTPUT}", (400, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv2.putText(combined_img, f"{OUTPUT}", (470, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Stream results
        if True:
            cv2.imshow("YOLOv8 Tracking", combined_img)
            cv2.waitKey(1)  # 1 millisecond

        vid_writer.write(combined_img)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        # time.sleep(0.1)

    vid_writer.release()
    cv2.destroyAllWindows()


0: 128x1024 (no detections), 101.0ms
Speed: 2.5ms preprocess, 101.0ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 85.2ms
Speed: 1.2ms preprocess, 85.2ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 65.2ms
Speed: 1.0ms preprocess, 65.2ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 65.4ms
Speed: 1.5ms preprocess, 65.4ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 62.5ms
Speed: 0.0ms preprocess, 62.5ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 76.3ms
Speed: 2.5ms preprocess, 76.3ms inference, 0.6ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 71.8ms
Speed: 1.0ms preprocess, 71.8ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 65.6ms
Speed: 1.2ms preprocess, 65.6ms inference, 1.3ms postproce

BREAK: 98

BREAK: 98



0: 128x1024 2 humanss, 64.3ms
Speed: 1.0ms preprocess, 64.3ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 63.0ms
Speed: 0.0ms preprocess, 63.0ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 64.1ms
Speed: 0.5ms preprocess, 64.1ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 1 humans, 65.5ms
Speed: 1.0ms preprocess, 65.5ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 1 humans, 63.3ms
Speed: 1.2ms preprocess, 63.3ms inference, 1.1ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 61.6ms
Speed: 1.1ms preprocess, 61.6ms inference, 1.5ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 63.6ms
Speed: 1.3ms preprocess, 63.6ms inference, 1.1ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 64.1ms
Speed: 1.0ms preprocess, 64.1ms inference, 0.0ms postprocess per imag

BREAK: 143

BREAK: 143



0: 128x1024 2 humanss, 67.3ms
Speed: 0.0ms preprocess, 67.3ms inference, 1.3ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 66.3ms
Speed: 1.2ms preprocess, 66.3ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)



...

0: 128x1024 2 humanss, 66.9ms
Speed: 0.0ms preprocess, 66.9ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 67.3ms
Speed: 2.5ms preprocess, 67.3ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 1024)



BREAK: 143



0: 128x1024 2 humanss, 68.9ms
Speed: 0.5ms preprocess, 68.9ms inference, 0.5ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 65.6ms
Speed: 1.0ms preprocess, 65.6ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)



BREAK: 143



0: 128x1024 2 humanss, 65.4ms
Speed: 1.1ms preprocess, 65.4ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 67.6ms
Speed: 0.0ms preprocess, 67.6ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 65.7ms
Speed: 1.0ms preprocess, 65.7ms inference, 1.5ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 66.6ms
Speed: 2.0ms preprocess, 66.6ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 65.3ms
Speed: 0.0ms preprocess, 65.3ms inference, 1.5ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 66.1ms
Speed: 1.0ms preprocess, 66.1ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 64.3ms
Speed: 1.4ms preprocess, 64.3ms inference, 1.0ms postprocess per image at shape (1, 3, 128, 1024)

0: 128x1024 2 humanss, 65.7ms
Speed: 0.0ms preprocess, 65.7ms inference, 0.0ms postprocess per im